# Configure structure based analysis for `dms-viz`

Imports:

In [ ]:
import gzip
import os
import requests
import subprocess
import textwrap
import warnings

import Bio.PDB.PDBParser
import Bio.PDB.Polypeptide

import pandas as pd

Define variables. This next cell is tagged `parameters` for `papermill` parameterization:

In [ ]:
pdb_id = None
phenotypes_csv = None
site_numbering_map = None
dms_viz_json = None
dms_viz_phenotypes = None
pdb_file = None
dms_viz_subdir = None
chains = None

Read the phenotypes: 

In [ ]:
phenotypes = pd.read_csv(phenotypes_csv)

wts = phenotypes[["site", "wildtype"]].drop_duplicates()
# I've got insertion at 16a so I have to drop and convert to numeric
# Convert to numeric, setting non-numeric to NaN
wts['site'] = pd.to_numeric(wts['site'], errors='coerce')

# Drop rows with NaN in 'site'
wts = wts.dropna(subset=['site'])

# Convert from float to int if needed
wts['site'] = wts['site'].astype(int)

Get the biological assembly (see https://pdb101.rcsb.org/learn/guide-to-understanding-pdb-data/biological-assemblies#Anchor-download) as the crystallographic unit doesn't correspond to that:

In [ ]:
r = requests.get(f"https://files.rcsb.org/download/{pdb_id}.pdb1.gz")
assert r.status_code == 200
pdb_content = gzip.decompress(r.content).decode("utf-8")
with open(pdb_file, "w") as f:
    f.write(pdb_content)

Check the sites mismatched between the sitemap and the protein structure in terms of residue identity:

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    pdb_obj = Bio.PDB.PDBParser().get_structure(id=pdb_id, file=pdb_file)[0]

records = []
for chain in ["A","B","C"]:
    for res in pdb_obj[chain].get_residues():
        if not res.id[0].isspace():
            continue
        aa = Bio.PDB.Polypeptide.protein_letters_3to1[res.resname]
        r = res.id[1]
        records.append((chain, r, aa))
pdb_df = pd.DataFrame(records, columns=["chains", "site", "pdb_aa"])

mismatched_sites = wts.merge(pdb_df, how="left")

print(
    f"Of {len(wts)} sites, {len(mismatched_sites.query('wildtype == pdb_aa'))} match, "
    f"{len(mismatched_sites.query('pdb_aa.isnull()'))} are missing from PDB, and "
    f"{len(mismatched_sites.query('pdb_aa.notnull()').query('wildtype != pdb_aa'))} differ."
)

print("Sites that differ:")
display(mismatched_sites.query("pdb_aa.notnull() and (wildtype != pdb_aa)").reset_index(drop=True))

Write CSV with phenotypes:

In [ ]:
phenotypes = (
    phenotypes
    .assign(mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"])
    .rename(columns={c: c.replace(" ", "_") for c in phenotypes.columns})
)

print(f"Phenotypes has following columns: {phenotypes.columns.tolist()}")

phenotypes.to_csv(dms_viz_phenotypes, index=False)

Run [configure-dms-viz](https://dms-viz.github.io/dms-viz-docs/preparing-data/command-line-api/).
First, set up some options:

In [ ]:
phenotype_cols = {
    # phenotype columns and additional arguments to `configure-dms-viz`
    "R568_2E7": ["--floor", "True", "--summary-stat", "sum"]
}

# additional tooltips to show
tooltip_cols = {
    c: c.replace("_", " ")
    for c in list(phenotype_cols) + ["mutation", "ACE2_binding", "spike_mediated_entry"]
}

assert set(tooltip_cols).issubset(phenotypes.columns), f"{tooltip_cols=}\n{phenotypes.columns=}"

filter_limits = {
    "spike_mediated_entry": [float(phenotypes["spike_mediated_entry"].min()), -3, 0],
    "ACE2_binding": [float(phenotypes["ACE2_binding"].min()), -3, 0],
}
filter_cols = list(filter_limits)

Now make the JSONs for each phenotype, and then combine them:

In [ ]:
pheno_jsons = []
for pheno_col, pheno_args in phenotype_cols.items():
    pheno_json = os.path.join(dms_viz_subdir, f"{pheno_col}.json")
    print(f"Writing phenotype {pheno_col} to {pheno_json}")
    cmds = [
        "configure-dms-viz", "format",
        "--name", pheno_col,
        "--input", dms_viz_phenotypes,
        "--metric", pheno_col,
        "--structure", pdb_file,
        "--sitemap", site_numbering_map,
        "--included-chains", " ".join(chains.split(",")),
        "--tooltip-cols", str({k: v for (k, v) in tooltip_cols.items() if k != pheno_col}),
        "--alphabet", "RKHDEQNSTYWFAILMVGPC-",
        "--output", pheno_json,
        "--colors" , "#F02D3A",
        "--title", f"Effects of mutations to XBB.1.5 spike on {pheno_col} escape",
        "--description", pheno_col,
        *pheno_args,
    ]
    pheno_filter_cols = {c: c for c in filter_cols if c != pheno_col}
    if pheno_filter_cols:
        cmds += ["--filter-cols", str(pheno_filter_cols)]
        cmds += ["--filter-limits", str({c: filter_limits[c] for c in pheno_filter_cols})]
    result = subprocess.run(cmds, capture_output=True, text=True)
    if result.returncode != 0:
        raise ValueError(f"Error running configure-dms-viz:\n\n{result.args=}\n\n{result.stdout=}\n\n{result.stderr=}\n")
    pheno_jsons.append(pheno_json)

markdown_description = os.path.join(dms_viz_subdir, "description.md")
with open(markdown_description, "w") as f:
    f.write(
        textwrap.dedent(
            """\
            # Effects of mutations to XBB.1.5 spike on escape from R568-2E7 mAb.
            """
        )
    )

print(f"Concatenating phenotype JSONs to {dms_viz_json}")
subprocess.run(
    [
        "configure-dms-viz", "join",
        "--input", ", ".join(pheno_jsons),
        "--output", dms_viz_json,
        "--description", markdown_description,
    ],
    check=True,
)